# Sistema de recomendacion basado en contenido

In [6]:
import numpy as np
import pandas as pd
df_movies = pd.read_csv('C:\\Users\ManVi\Desktop\\repo\movies_dataset_clean1_1.csv')

In [254]:
df_movies.head(2)


,Unnamed: 0,belongs_to_collection,budget,id,original_language,overview,popularity,release_date,revenue,runtime,...,vote_count,Genres,pro_comp1,pro_comp2,country,language,release_year,return,director,actors
0,0,Toy Story Collection,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,...,5415.0,"['Animation', 'Comedy', 'Family']",Pixar Animation Studios,NaN,United States of America,English,1995,12.451801,John Lasseter,"['Tom Hanks', 'Tim Allen', 'Don Rickles']"
1,1,NaN,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,...,2413.0,"['Adventure', 'Fantasy', 'Family']",TriStar Pictures,Teitler Film,United States of America,Français,1995,4.043035,Joe Johnston,"['Robin Williams', 'Jonathan Hyde', 'Kirsten D..."


In [263]:
df_movies = pd.read_csv('movies_dataset_clean1_1.csv')



#### se trabajara el modelo de recomendacion a partir de las columnas `Genres` y `overview`

In [275]:
#importo los modulos de sk_learn necesarios para el proyecto
#TfidfVectorizer: para calcular el TD-IDF de las columnas overview y genres.
#cosine_similarity: para calcular la matriz de similaridad de las peliculas

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargar el dataset
df_movies = pd.read_csv('movies_dataset_clean1_1.csv')

#no uso todos los registros,no tengo los suficientes recursos para
#procesarlos todos, se deberian usar mas datos para obtener mejores
#recomendaciones
df_movies = df_movies.loc[:2300]
#relleno los nulos para el campo overview
df_movies['overview'] = df_movies['overview'].fillna('')


# Crear vectores TF-IDF
# uno para overview, y otro para genre
tfidf_vectorizer = TfidfVectorizer(min_df=5 ,max_df=0.7,token_pattern= r'\b[a-zA-Z]\w+\b' ,stop_words='english')
tfidf_vector_genres = tfidf_vectorizer.fit_transform(df_movies['Genres'])
tfidf_vector_overview = tfidf_vectorizer.fit_transform(df_movies['overview'])

tfidf_df_genres = pd.DataFrame(tfidf_vector_genres.toarray())
tfidf_df_overview = pd.DataFrame(tfidf_vector_overview.toarray())
tfidf_df_genres = tfidf_df_genres.astype('float16')
tfidf_df_overview = tfidf_df_overview.astype('float16')

# Calcular la matriz de similitud coseno para los dos arrays de vectores
cosine_sim_genres = cosine_similarity(tfidf_df_genres.values)
cosine_sim_overview = cosine_similarity(tfidf_df_overview.values)

# Función de recomendación
def get_recommendations(title, cosine_sim):
    '''
    def get_recommendation:
    
    title ---> titulo de una pelicula, como aparece en el dataset
    cosine_sim ---> matriz de similaridad. Puedes tomar 2 valores:
    
    cosine_sim_genres: se hace la recomendacion en base a los generos
    cosine_sim_overview: se hace la recomendacion en base a la overview
    
    return: lista con las primeras 5 peliculas mas similares a la dada
    
    '''
    idx = df_movies.index[df_movies['title'] == title].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    return df_movies['title'].iloc[movie_indices]



In [248]:
cosine_sim_overview

array([[1.        , 0.01898132, 0.        , ..., 0.        , 0.01426819,
        0.02798732],
       [0.01898132, 1.        , 0.05474509, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.05474509, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.03840896,
        0.        ],
       [0.01426819, 0.        , 0.        , ..., 0.03840896, 1.        ,
        0.01877063],
       [0.02798732, 0.        , 0.        , ..., 0.        , 0.01877063,
        1.        ]])

In [164]:
df_movies['title']

0                          Toy Story
1                            Jumanji
2                   Grumpier Old Men
3                  Waiting to Exhale
4        Father of the Bride Part II
                    ...             
2596                   Summer of Sam
2597                  Broken Vessels
2598         Les Amants du Pont-Neuf
2599    Late August, Early September
2600                    American Pie
Name: title, Length: 2601, dtype: object

In [274]:
# Ejemplo de recomendación
movie_title = 'Toy Story'
print(f"Recomendaciones para '{movie_title}':")
print(list(get_recommendations(movie_title, cosine_sim_genres)))


Recomendaciones para 'Toy Story':
['Oliver & Company', 'A Close Shave', 'The Wrong Trousers', 'Meet the Deedles', 'The Great Mouse Detective']


In [270]:
# Ejemplo de recomendación
movie_title = 'Toy Story'
print(f"Recomendaciones para '{movie_title}':")
print(get_recommendations(movie_title, cosine_sim_overview))


Recomendaciones para 'Toy Story':
1938             Condorman
485                 Malice
1039         The Sunchaser
448      For Love or Money
749     I Shot Andy Warhol
Name: title, dtype: object


In [276]:
# GUARDO ESTOS DATOS PARA USARLOS EN LA API
data = pd.DataFrame(cosine_sim_genres)

data.to_csv('movie_simil_genres.csv')